# Convert MONAI models to OpenVINO

This notebook loads MONAI models listed on https://docs.monai.io/en/latest/networks.html#nets and converts them to ONNX and IR. Models list is current as of November 11, 2021 with MONAI 0.7.0

This notebook converts all listed models except:
- SENet: SENet architecture is validated by converting SENet154 instead of the generic SENet model
- Tranchex: Tranchex has a multimodal architecture that requires specific arguments. Tranchex has been manually validated and code will be added soon
- Netadapter: This is not a model, but a wrapper to replace a layer with a PyTorch model layer
- VitAutoEnc: does not exist in Monai 0.7.0
- TorchVisionFullyConvModel: deprecated (according to MONAI docs) in favor of TorchVisionFCModel

Running the notebook requires installing monai, openvino-dev and a few other dependencies. Executing the next cell installs these packages in the current Python/Jupyter environment.

In [ ]:
%pip install --quiet --upgrade monai openvino-dev torch torchvision onnx einops ipywidgets

## Imports

In [ ]:
import inspect
import logging
import os
import time
from pathlib import Path

import monai.networks.nets as nets
import numpy as np
import torch
from IPython.display import Markdown
from openvino.inference_engine import IECore

## Settings

Set `ONNX_DIR` and `IR_DIR` to the names of the directories where the converted ONNX and IR models should be saved. These directories will be created if they do not exist.

In [ ]:
ONNX_DIR = "onnx_models"
IR_DIR = "ir_models"

Path(ONNX_DIR).mkdir(exist_ok=True)
Path(IR_DIR).mkdir(exist_ok=True)

`default_args` defines default argument for models, `network_args` overrides some default arguments for specific networks. Network dimensions are by default set to `1, in_channels, 128,128,128` for 3D networks and `1, in_channels, 128,128` for 2D networks. Some networks need a specific input shape, which is defined in `network_dims`.

In [ ]:
# These values work - but will likely not result in a good model
# For testing purposes only

default_args = {
    "channels": [2, 2],
    "depth": 5,
    "img_size": 128,
    "in_channels": 1,
    "in_shape": (1, 128, 128),
    "model_name": "efficientnet-b0",
    "num_channel_initial": 16,
    "out_channels": 1,
    "spatial_dims": 2,
    "strides": ((1, 2), 1, 1),
}

network_args = {
    "AutoEncoder": {"strides": [1, 1]},
    "Classifier": {"classes": 2},
    "DynUNet": {"kernel_size": ((1, 2), 1, 1), "upsample_kernel_size": (1, 1)},
    "EfficientNet": {"blocks_args_str": ["r1_k3_s11_e1_i32_o16_se0.25"]},
    "FullyConnectedNet": {"in_channels": 16384, "hidden_channels": []},
    "Generator": {"start_shape": (64, 8, 8), "latent_shape": (128, 128)},
    "GlobalNet": {"image_size": (128, 128)},
    "LocalNet": {"extract_levels": [1]},
    "Regressor": {"out_shape": (128, 128)},
    "SegResNetVAE": {"input_image_size": [1, 128, 128]},
    "TorchVisionFCModel": {"pool": None},
    "UNETR": {"spatial_dims": 2},
    "VarAutoEncoder": {"strides": [1, 1], "latent_size": 2},
    "ViT": {"img_size": (1, 128, 128), "patch_size": (1, 16, 16)},
}

network_dims = {
    "FullyConnectedNet": [1, 128 * 128],
    "Generator": (1, 1, 1, 128, 128),
    "HighResNet": (1, 1, 64, 64, 64),
    "Regressor": (1, 1, 128, 128),
    "TorchVisionFCModel": (1, 3, 256, 256),
    "UNETR": (1, 1, 128, 128),
    "ViT": (1, 1, 1, 128, 128),
}

`monai_net_names` is a string of models ot convert, separated by spaces.

In [ ]:
monai_net_names = "AHNet DenseNet DenseNet121 DenseNet169 DenseNet201 DenseNet264 EfficientNet EfficientNetBN EfficientNetBNFeatures SegResNet SegResNetVAE SENet154 SEResNet50 SEResNet101 SEResNet152 SEResNext50 SEResNext101 HighResNet DynUNet UNet UNETR BasicUNet VNet RegUNet GlobalNet LocalNet AutoEncoder VarAutoEncoder ViT FullyConnectedNet Generator Regressor Classifier Discriminator Critic TorchVisionFCModel"

## Convert to ONNX

In [ ]:
%%capture --no-stdout
logging.basicConfig(filename="monai_models.log", level=logging.INFO)
monai_nets = monai_net_names.split(" ")
monai_nets = [net_name.strip() for net_name in monai_nets]
num = len(monai_nets)

for i, net_name in enumerate(sorted(monai_nets)):
    print(f"[{i+1}/{num}] {net_name}", end=" ")
    net = getattr(nets, net_name)
    kwargs = {}
    # Override default arguments if necessary
    if net_name in network_args:
        for key, value in network_args[net_name].items():
            kwargs[key] = value

    # Find parameters to use with this network
    argset = set()
    for arg in inspect.signature(net).parameters.values():
        if arg.name == "pool":
            argset.add(arg.name)
        elif arg.default == inspect._empty and arg.name not in ["args", "kwargs"]:
            argset.add(arg.name)

    # Some networks require default arguments only defined in a parent class
    for arg in inspect.signature(net.__bases__[0]).parameters.values():
        if (
            arg.default == inspect._empty
            and arg.name not in ["args", "kwargs"]
            and arg.name not in inspect.signature(net).parameters
            and arg.name in {"in_channels", "out_channels", "spatial_dims"}
        ):
            argset.add(arg.name)
    kwargs.update({arg: default_args[arg] for arg in argset if arg not in kwargs})

    # VarAutoEncoder inherits from AutoEncoder and should support in_channels, but does not.
    if net_name == "VarAutoEncoder":
        kwargs.pop("in_channels")

    # Load the MONAI network
    try:
        monainet = net(**kwargs)
    except:
        print("loading failed")
        raise

    # Determine input shape for ONNX export
    input_shape = list(list(monainet.parameters())[0].shape)
    in_channels = input_shape[1] if len(input_shape) > 3 else input_shape[0]
    if net_name in network_dims:
        new_input_shape = network_dims[net_name]
    elif len(input_shape) == 5:
        new_input_shape = (1, in_channels, 128, 128, 128)
    elif len(input_shape) == 3 or len(input_shape) == 4:
        new_input_shape = (1, in_channels, 128, 128)

    # Log the model with the arguments, and input shape
    # for example SEResNet50(in_channels=1, spatial_dims=2)
    model_args = ", ".join(f"{key}={value}" for (key, value) in kwargs.items())
    logging.info(rf"{net_name}({model_args}) input shape: {new_input_shape}")

    # Export to ONNX
    dummy_input = torch.randn(new_input_shape)
    print(new_input_shape, end=" ")
    try:
        torch.onnx.export(
            monainet,
            dummy_input,
            f"{ONNX_DIR}/{net_name}.onnx",
            opset_version=12,
            do_constant_folding=False,
        )
        print("succeeded")
    except:
        print("converting failed")
        raise
    finally:
        del monainet

## Convert to OpenVINO IR

Convert all ONNX models to IR format. Model Optimizer output is not shown in the notebook, but added to mo_log.txt.

In [ ]:
onnx_models = list(Path(ONNX_DIR).glob("*.onnx"))
log = Path("mo_log.txt")
num = len(onnx_models)
with open(log, mode="a") as logfile:
    for i, onnx_model_path in enumerate(onnx_models):
        ir_path = Path(IR_DIR) / onnx_model_path.with_suffix(".xml").name
        if not ir_path.exists():
            print(f"[{i+1}/{num}] Converting {ir_path.stem}...")
            mo_result = !mo --data_type FP16 --input_model $onnx_model_path --output_dir $IR_DIR
            logfile.writelines(mo_result.get_nlstr())
        else:
            print(f"[{i+1}/{num}] Skipping {ir_path.stem} (already exists)")

## Verify that all models converted to ONNX and IR

In [ ]:
found_missing = False

for net in monai_nets:
    onnx_path=Path(f"{ONNX_DIR}/{net_name}.onnx")
    if not onnx_path.exists():
        print(f"{onnx_path} does not exist")
        found_missing = True

for onnx_model_path in onnx_models:
    ir_path = Path(IR_DIR) / onnx_model_path.with_suffix(".xml").name
    if not ir_path.exists():
        print(f"{ir_path} does not exist")
        found_missing = True
        
if found_missing:
    raise Exception("Not all MONAI models converted successfully")
else:
    print("All MONAI models converted to ONNX and IR!")

## Test Inference

In [ ]:
# Exclude Regressor. With the current network parameters it crashes Jupyter on CPU,
# but works fine on GPU
ignore_models = ["Regressor"]
ir_models = Path(IR_DIR).glob("*.xml")
for ir_path in ir_models:
    if ir_path.stem not in ignore_models:
        print(ir_path, end=" ")
        ie = IECore()
        net = ie.read_network(model=ir_path)
        exec_net = ie.load_network(network=net, device_name="CPU")
        input_layer = next(iter(net.input_info))
        output_layer = next(iter(net.outputs))

        input_dims = net.input_info[input_layer].tensor_desc.dims
        print(input_dims, end=" ")
        input_data = np.random.randint(low=0, high=255, size=input_dims, dtype=np.uint8).astype(
            np.float32
        )
        result = exec_net.infer({input_layer: input_data})
        print("succeeded" if result.get(output_layer) is not None else "failed")
        del net
        del exec_net
        del ie
        del input_data

## Benchmark

Benchmark all models. This will take quite a long time, and not give very accurate output. For testing purposes only.

In [ ]:
def benchmark_model(model_path: os.PathLike,
                    device: str = "CPU",
                    seconds: int = 60, api: str = "async",
                    batch: int = 1, 
                    cache_dir="model_cache"):
    ie = IECore()
    model_path = Path(model_path)
    if ("GPU" in device) and ("GPU" not in ie.available_devices):
        raise ValueError(f"A GPU device is not available. Available devices are: {ie.available_devices}")
    else:
        benchmark_command = f"benchmark_app -m {model_path} -d {device} -t {seconds} -api {api} -b {batch} -cdir {cache_dir}"
        display(Markdown(f"**Benchmark {model_path.name} with {device} for {seconds} seconds with {api} inference**"));
        display(Markdown(f"Benchmark command: `{benchmark_command}`"));

        benchmark_output = %sx $benchmark_command
        benchmark_result = [line for line in benchmark_output
                            if not (line.startswith(r"[") or line.startswith("  ") or line == "")]
        print("\n".join(benchmark_result))
        print()
        if "MULTI" in device:
            devices = device.replace("MULTI:","").split(",")
            for single_device in devices:
                print(f"{single_device} device: {ie.get_metric(device_name=single_device, metric_name='FULL_DEVICE_NAME')}")
        else:
            print(f"Device: {ie.get_metric(device_name=device, metric_name='FULL_DEVICE_NAME')}")

In [ ]:
# ignore_models = ["Regressor"]
# ir_models = Path(IR_DIR).glob("*.xml")
# for ir_path in ir_models:
#     if ir_path.stem not in ignore_models:
#         benchmark_model(ir_path, seconds=15)
#         time.sleep(15)